<a href="https://colab.research.google.com/github/liuchiente/AIOTAtNCHU/blob/main/Ptt_Roach.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PTT爬蟲程式

---
本程式主要用簡單的Python開發出爬蟲工具,會依照指定的條件,到某個PTT版上抓取新聞文章回來,再用CKIP解析文章內容,分析出文章裡面的人名、地名等資訊。

p.s.　道德勸說一下，雖然程式是死的，但請記得透過程式去抓資料的時間相當短，量也不小，要記得控制好抓的文章數和抓取時間，避免對網站造成龐大負擔。

by PTT的老鄉民 :(


#切換路徑到指定資料夾


In [ ]:
sys_dir = "/content/drive/My Drive/Colab Notebooks/pttroach/"
#連接並mount自己的雲端硬碟(點選跑出網址複製最後的授權碼貼上並執行)
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
#切換執行目錄
import os
if not os.path.isdir(sys_dir):
    os.mkdir(sys_dir)
os.chdir(sys_dir)

#引入套件

In [ ]:
! pip install fake_useragent

  Created wheel for fake-useragent: filename=fake_useragent-0.1.11-cp37-none-any.whl size=13485 sha256=dffe9bb5c2fc0e6e31ba227079cd08e37e08ccda70413b62693fa5e1284f49c5
  Stored in directory: /root/.cache/pip/wheels/5e/63/09/d1dc15179f175357d3f5c00cbffbac37f9e8690d80545143ff
Successfully built fake-useragent


In [ ]:
#引入套件
import requests
import re
import time
import json
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import datetime

#定義基本函式庫

In [ ]:
#隨機產生User angent,模擬瀏覽器行為,避免被阻擋
def set_header_user_agent():
    user_agent = UserAgent()
    return user_agent.random

#檢查是不是合法的網址
def check_url(url):
    regex = re.compile(
        r'^(?:http|ftp)s?://' # http:// or https://
        r'(?:(?:[A-Z0-9](?:[A-Z0-9-]{0,61}[A-Z0-9])?\.)+(?:[A-Z]{2,6}\.?|[A-Z0-9-]{2,}\.?)|'
        r'localhost|' #localhost...
        r'\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3})' # ...or ip
        r'(?::\d+)?' # optional port
        r'(?:/?|[/?]\S+)$', re.IGNORECASE)
    return re.match(regex, url)

#查詢資料
def req_url(link,param):
    user_agent = set_header_user_agent()
    req=requests.get(link,headers={'user-agent': user_agent,'cookie': 'over18=1'},params=param)
    #debug 網址
    #print(req.url)
    rslt=BeautifulSoup(req.text,'lxml')
    return rslt

#設定查詢條件

In [ ]:
#基本變數
q="%5B新聞%5D"
#是否停止
to_be_contiunted=True
#頁數
page=0
#查詢結果
r_list=[]
#現在時間
today = datetime.date.today()
#開始抓文章時間,預設從上個月1號開始
#first_day = today.replace(day=1,month=today.month-1)
first_day = today - datetime.timedelta(days=1)
#預設為空
r_articles=[]
if os.path.isfile('ptt_roach.json'):
  r_json_f=open('ptt_roach.json', "r")
  r_articles=json.loads(r_json_f.read())
#最後一篇文章日期
#if len(r_articles)>0:
  #articles_last=r_articles[-1]
  #print(articles_last)
  #first_day=datetime.datetime.strptime(articles_last['d'], "%Y/%m/%d")
#結束時間,預設抓一天
#last_day = first_day + datetime.timedelta(days=1)
r_articles.clear()
#last_day = today - datetime.timedelta(days=1)
last_day = today

#查詢看板列表

In [ ]:
to_be_contiunted=True
page=0
#只抓出特定日期區間範圍的文章
while to_be_contiunted:
    page=page+1
    rq=req_url('https://www.ptt.cc/bbs/Gossiping/search',{'q':'[新聞]','page':page})
    req_articles=rq.findAll('div',attrs = {'class' : 'r-ent'})
    for req_article in req_articles:
        req_date_s=req_article.find('div',attrs = {'class' : 'date'}).text.strip().zfill(5)
        #轉出時間
        req_date=datetime.datetime.strptime(str(today.year)+"/"+req_date_s, "%Y/%m/%d")
        if req_date.date() >= first_day and req_date.date() < last_day:
          r_link=req_article.find('a').get("href")
          if req_article.find('a').text.startswith('[新聞]') and not (r_link in r_list):
            #print(req_date_s+" "+req_article.find('a').text)
            r_art=dict()
            r_art['l']=r_link
            r_art['d']=datetime.datetime.strftime(req_date,"%Y/%m/%d")
            r_list.append(r_link)
        elif req_date.date() < first_day:
          to_be_contiunted=False
          break
    time.sleep(1)
print(r_list)

['/bbs/Gossiping/M.1616169418.A.843.html', '/bbs/Gossiping/M.1616169098.A.1B1.html', '/bbs/Gossiping/M.1616167476.A.CE4.html', '/bbs/Gossiping/M.1616167288.A.7BB.html', '/bbs/Gossiping/M.1616166721.A.98A.html', '/bbs/Gossiping/M.1616166455.A.EAF.html', '/bbs/Gossiping/M.1616166185.A.31E.html', '/bbs/Gossiping/M.1616165189.A.51B.html', '/bbs/Gossiping/M.1616164941.A.637.html', '/bbs/Gossiping/M.1616164389.A.407.html', '/bbs/Gossiping/M.1616164085.A.195.html', '/bbs/Gossiping/M.1616164024.A.FA6.html', '/bbs/Gossiping/M.1616163950.A.F89.html', '/bbs/Gossiping/M.1616163518.A.EEA.html', '/bbs/Gossiping/M.1616163485.A.062.html', '/bbs/Gossiping/M.1616162361.A.AAD.html', '/bbs/Gossiping/M.1616162268.A.2AD.html', '/bbs/Gossiping/M.1616162248.A.99A.html', '/bbs/Gossiping/M.1616162098.A.337.html', '/bbs/Gossiping/M.1616161966.A.366.html', '/bbs/Gossiping/M.1616161441.A.FED.html', '/bbs/Gossiping/M.1616161261.A.7EF.html', '/bbs/Gossiping/M.1616161251.A.964.html', '/bbs/Gossiping/M.1616160914.A.6E

#儲存文章清單

In [ ]:
print(len(r_list))
print(r_list[-1])
#寫入JSON檔案
with open('ptt_roach.json', 'w', encoding='utf-8') as f:
    json.dump(r_list, f, ensure_ascii=False, indent=4)

160
/bbs/Gossiping/M.1616084624.A.C1D.html


#將文章各自爬回

In [ ]:
#上次爬文結果
r_article_contents=[]
if os.path.exists('ptt_kerker_articles.json'):
  rl_json_f=open('ptt_kerker_articles.json', "r")
  r_article_contents=json.loads(rl_json_f.read())
#文章列表清單
rc_json_f=open('ptt_roach.json', "r")
rr_list=json.loads(rc_json_f.read())

160
0


In [ ]:
#將文章清單取回
for ra_url in rr_list[0:5]:
    r_article=req_url("https://www.ptt.cc"+ra_url,{})
    r_push=r_article.findAll('div',attrs = {'class' : ['push','richcontent','article-metaline','article-metaline-right']})
    for rp in r_push:
        rp.decompose()
    r_ah=r_article.findAll('a')
    for rah in r_ah:
        rah.decompose()
    r_spa=r_article.findAll('span',attrs = {'class' : ['f2','f6','b6','b7','b3','f1','article-meta-tag','article-metaline','article-meta-value']})
    for rspa in r_spa:
        rspa.decompose()
    r_info=r_article.find('div',attrs = {'id' : 'main-content'})
    try:
      r_article_content={}
      r_article_content['id']=len(r_article_contents)+1
      r_article_content['content']=r_info.text
      r_article_contents.append(r_article_content)
      rr_list.remove(ra_url)
    except Exception as e:
      print(e)
      print(r_article)
    time.sleep(2)

#儲存收集回來的文章

In [ ]:
#剩餘未處理
print(len(rr_list))
#已經取得文章
print(len(r_article_contents))

9
151


In [ ]:
#剩餘未處理
print(len(rr_list))
#已經取得文章
print(len(r_article_contents))
#寫入JSON檔案
with open('ptt_kerker_articles.json', 'w', encoding='utf-8') as f:
    json.dump(r_article_contents, f, ensure_ascii=False, indent=4)
#寫入JSON檔案
with open('ptt_kerker.json', 'w', encoding='utf-8') as f:
    json.dump(rr_list, f, ensure_ascii=False, indent=4)

9
151


In [ ]:
rd_json_f=open('ptt_kerker_articles.json', "r")
r_article_contents=json.loads(rd_json_f.read())
r_article_contents_list=[]
for r_article_content in r_article_contents:
  r_article_content['id']=r_article_content['id']+70000
  r_article_content['content']=r_article_content['content'].replace('\n','').replace('\t','').strip()
  tidx=r_article_content['content'].find('新聞標題')
  title=''
  if  tidx>0:
    title=r_article_content['content'][tidx:101]
  else:
    title=r_article_content['content'][0:101]
  r_article_content['title']=title
  r_article_contents_list.append(r_article_content)

print(r_article_contents_list[-1])


{'id': 70151, 'content': '1.媒體來源:中央廣播電台2.記者署名:撰稿編輯：吳寧康3.完整新聞標題:EMA肯定安全有效 歐盟國家恢復接種AZ疫苗4.完整新聞內文:在歐盟藥品管理局(EMA)18日表示，阿斯特捷利康(AstraZeneca)的2019年冠狀病毒疾病(COVID-19)疫苗是「安全有效」，而且和更高的血栓風險無關後，歐洲主要國家說，他們將會恢復對這支AZ疫苗的接種。由於有AZ疫苗的接種者出現血管出血(bleeding)、血栓(blood clots)與低血小板數量(low platelet counts)等嚴重不良副作用，因此包括歐盟最大成員國的德國、法國和義大利在內，已有多國宣佈暫停接種AZ疫苗。而歐盟藥品管理局發表這項備受矚目的宣佈之際，正值世界衛生組織(WHO)和英國衛生監管當局都已表示，AZ疫苗是安全的，而且對幾個正面臨確診病例令人憂心升高的國家來說，不接種疫苗的風險反而更高。在經過旗下一個安全委員會的調查後，歐盟藥品管理局執行主任庫克(Emer Cooke)18日表示，委員會已取得一項明確的科學結論：這是一支安全且有效的疫苗，而且委員會還得到結論，這支疫苗和血栓栓塞事件或血栓的總體風險上升無關。然而，庫克也指出，他們無法「確切排除」血栓事件和疫苗之間的關聯。在歐盟藥品管理局發表聲明後，包括德國、法國、西班牙、義大利、荷蘭、葡萄牙、立陶宛、拉脫維亞、斯洛維尼亞和保加利亞也都表示，將很快恢復接種AZ疫苗。而在15日暫停使用AZ疫苗的法國指出，將從19日起恢復施打這支疫苗，法國總理卡斯特克斯(Jean Castex)將會是接種者之一。此外，法國18日也成為最新一個加強防疫限制的國家，宣佈對巴黎和其他幾個地區實施一個月的有限封城，試圖擊退已導致醫院不堪重負的第三波疫情感染。5.完整新聞連結 (或短網址):6.備註:歐洲好多國家要恢復AZ施打了不過一開始帶頭的北歐好像還在觀望?台灣也快要開始打了 你各位OK嗎--', 'title': '新聞標題:EMA肯定安全有效 歐盟國家恢復接種AZ疫苗4.完整新聞內文:在歐盟藥品管理局(EMA)18日表示，阿斯特捷利康(AstraZe'}


In [ ]:
with open('ptt_kerker_articles_final.json', 'w', encoding='utf-8') as f:
    json.dump(r_article_contents_list, f, ensure_ascii=False, indent=4)

#CKIP

In [ ]:
!pip install ckiptagger
!pip install tensorflow
!pip install gdown

#CKIP語料

In [ ]:
# -*- coding: utf-8 -*-
from ckiptagger import data_utils
data_utils.download_data_gdown("./")

Downloading...
From: https://drive.google.com/uc?id=1efHsY16pxK0lBD2gYCgCTnv1Swstq771
To: /content/drive/My Drive/Colab Notebooks/pttroach/data.zip
1.88GB [00:39, 47.2MB/s]


In [ ]:
# 將文章切出詞組,區分為地名、人名之類的
# -*- coding: utf-8 -*-
from ckiptagger import WS, POS, NER

rd_json_f=open('ptt_kerker_articles_final.json', "r")
r_article_contents_list=json.loads(rd_json_f.read())

r_article_contents_last=r_article_contents_list[-1]

text = r_article_contents_last['content']
ws = WS("./data")
pos = POS("./data")
ner = NER("./data")

ws_results = ws([text])
pos_results = pos(ws_results)
ner_results = ner(ws_results, pos_results)

for name in ner_results[0]:
    print(name)